## Telegram group chat data analysis

For downloading chat data, go to Settings > Advanced > Export Telegram data > Machine-readable JSON

For downloading for a chat individually, JSON format is not available (but there is a github issue on that, so hopefully it will be available soon)

In [ ]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
file = 'result.json' # You may change this if you changed the name of the file
with open(file) as file:
    dict_data = json.load(file)
dict_data # Data loaded to dictionary

In [ ]:
# Messages are extracted for the selected group
chat_name='test_name' # the name of the group chat
chat_data_list=dict_data['chats']['list']
for chat in chat_data_list:
    if chat['name']==chat_name:
        chat_data=chat
        break
chat_data_fil=[] # filter events like picture change or photos
# chat_members=[chat_data['messages'][0]['members']] # Initial members
# print("Chat members", chat_members)
for m in chat_data['messages']:
    if (m['type'] == 'message' and m['text']!=''):
        chat_data_fil.append(m)

In [ ]:
# Convert to dataframe
df_chat = pd.DataFrame.from_dict(chat_data_fil)
# Only want user and message, although more info like date could be kept
col_sel=['from','text']
df_chat=df_chat[col_sel]

In [ ]:
usernames=[]
messages=[]
for user, mes in tele.groupby(['from']):
    usernames.append(user)
    messages.append(mes['text'])
print("Users: ", usernames)

In [ ]:
# Stopwords are words commonly used in a language that do not provide much information
# For english, sklearn has a defined list, so stopwords='english' may be used
# For other languages or custom words, fill in the list
# E.g for Spanish
stopwords=['de', 'que', 'la', 'es', 'el', 'en','me', 'lo', 'un', 'los', 'una', 'han', 'las', 'al', 'se']

In [ ]:
# Function
# Get top words from user 
# Function adapted from Cristhian Boujon at Medium
# https://medium.com/@cristhianboujon/how-to-list-the-most-common-words-from-text-corpus-using-scikit-learn-dad4d0cab41d
def get_top_n_words(user, stopwords, n=None):
    user_index=usernames.index(user)
    corpus=[str (item) for item in messages[user_index]]
    vec = CountVectorizer(stop_words=stopwords).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
# Now we can get the top words for a user
get_top_n_words('someone', stopwords, n=100)